In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, precision_recall_curve
from sklearn.metrics import confusion_matrix, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from collections import Counter

import seaborn as sns

In [69]:
# train_df = pd.read_csv('train.csv')
train_df = pd.read_csv('train2.csv')
train_df = train_df.drop([' Fwd Header Length.1',' Label'],axis = 1)
train_df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,80,38308,1,1,6,6,6,6,6.000000,0.000000,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
1,389,479,11,5,172,326,79,0,15.636364,31.449238,...,4,32,0.0,0.0,0,0,0.0,0.0,0,0
2,88,1095,10,6,3150,3150,1575,0,315.000000,632.561635,...,3,32,0.0,0.0,0,0,0.0,0.0,0,0
3,389,15206,17,12,3452,6660,1313,0,203.058824,425.778474,...,10,32,0.0,0.0,0,0,0.0,0.0,0,0
4,88,1092,9,6,3150,3152,1575,0,350.000000,694.509719,...,2,32,0.0,0.0,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692698,53,32215,4,2,112,152,28,28,28.000000,0.000000,...,3,20,0.0,0.0,0,0,0.0,0.0,0,0
692699,53,324,2,2,84,362,42,42,42.000000,0.000000,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
692700,58030,82,2,1,31,6,31,0,15.500000,21.920310,...,0,32,0.0,0.0,0,0,0.0,0.0,0,0
692701,53,1048635,6,2,192,256,32,32,32.000000,0.000000,...,5,20,0.0,0.0,0,0,0.0,0.0,0,0


In [70]:
test_df = pd.read_csv('test.csv')
test_df = test_df.drop(['protocol','timestamp'], axis = 1)
test_df

,dst_port,flow_duration,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,fwd_pkt_len_mean,fwd_pkt_len_std,...,fwd_act_data_pkts,fwd_seg_size_min,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min
0,80,26870384,5,0,659,0,413,60,131.8,140.619202,...,4,20,0.0,0.0,0,0,7.291958e+06,10295440.45,21851909,9971
1,43908,372198,5,0,486,0,214,66,97.2,58.482134,...,1,20,0.0,0.0,0,0,1.575985e+05,158.50,157757,157440
2,80,27580343,5,0,827,0,581,60,165.4,207.812993,...,4,20,0.0,0.0,0,0,7.517380e+06,10594394.43,22500101,7682
3,80,27248690,5,0,627,0,381,60,125.4,127.821125,...,4,20,0.0,0.0,0,0,7.409746e+06,10452004.13,22191106,7534
4,80,28830565,5,0,791,0,545,60,158.2,193.413960,...,4,20,0.0,0.0,0,0,7.927495e+06,11165319.06,23717619,9795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12496,80,30027649,4,0,246,0,66,60,61.5,2.598076,...,3,20,0.0,0.0,0,0,1.500093e+07,14990575.50,29991510,10359
12497,80,2119685,4,0,246,0,66,60,61.5,2.598076,...,3,20,0.0,0.0,0,0,1.059838e+06,1047813.50,2107651,12024
12498,80,30021918,4,0,246,0,66,60,61.5,2.598076,...,3,20,0.0,0.0,0,0,1.500026e+07,14990399.00,29990661,9863
12499,80,30023465,4,0,246,0,66,60,61.5,2.598076,...,3,20,0.0,0.0,0,0,1.500112e+07,14991796.00,29992920,9328


### 컬럼 비교

In [6]:
train_df_list = train_df.columns
test_df_list = test_df.columns
print(len(train_df_list), len(test_df_list))
for i in range(len(train_df_list)):
    print(train_df_list[i],test_df_list[i])

77 77
 Destination Port dst_port
 Flow Duration flow_duration
 Total Fwd Packets tot_fwd_pkts
 Total Backward Packets tot_bwd_pkts
Total Length of Fwd Packets totlen_fwd_pkts
 Total Length of Bwd Packets totlen_bwd_pkts
 Fwd Packet Length Max fwd_pkt_len_max
 Fwd Packet Length Min fwd_pkt_len_min
 Fwd Packet Length Mean fwd_pkt_len_mean
 Fwd Packet Length Std fwd_pkt_len_std
Bwd Packet Length Max bwd_pkt_len_max
 Bwd Packet Length Min bwd_pkt_len_min
 Bwd Packet Length Mean bwd_pkt_len_mean
 Bwd Packet Length Std bwd_pkt_len_std
Flow Bytes/s flow_byts_s
 Flow Packets/s flow_pkts_s
 Flow IAT Mean flow_iat_mean
 Flow IAT Std flow_iat_std
 Flow IAT Max flow_iat_max
 Flow IAT Min flow_iat_min
Fwd IAT Total fwd_iat_tot
 Fwd IAT Mean fwd_iat_mean
 Fwd IAT Std fwd_iat_std
 Fwd IAT Max fwd_iat_max
 Fwd IAT Min fwd_iat_min
Bwd IAT Total bwd_iat_tot
 Bwd IAT Mean bwd_iat_mean
 Bwd IAT Std bwd_iat_std
 Bwd IAT Max bwd_iat_max
 Bwd IAT Min bwd_iat_min
Fwd PSH Flags fwd_psh_flags
 Bwd PSH Flags bwd

In [71]:
test_df.columns = train_df_list
test_df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,80,26870384,5,0,659,0,413,60,131.8,140.619202,...,4,20,0.0,0.0,0,0,7.291958e+06,10295440.45,21851909,9971
1,43908,372198,5,0,486,0,214,66,97.2,58.482134,...,1,20,0.0,0.0,0,0,1.575985e+05,158.50,157757,157440
2,80,27580343,5,0,827,0,581,60,165.4,207.812993,...,4,20,0.0,0.0,0,0,7.517380e+06,10594394.43,22500101,7682
3,80,27248690,5,0,627,0,381,60,125.4,127.821125,...,4,20,0.0,0.0,0,0,7.409746e+06,10452004.13,22191106,7534
4,80,28830565,5,0,791,0,545,60,158.2,193.413960,...,4,20,0.0,0.0,0,0,7.927495e+06,11165319.06,23717619,9795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12496,80,30027649,4,0,246,0,66,60,61.5,2.598076,...,3,20,0.0,0.0,0,0,1.500093e+07,14990575.50,29991510,10359
12497,80,2119685,4,0,246,0,66,60,61.5,2.598076,...,3,20,0.0,0.0,0,0,1.059838e+06,1047813.50,2107651,12024
12498,80,30021918,4,0,246,0,66,60,61.5,2.598076,...,3,20,0.0,0.0,0,0,1.500026e+07,14990399.00,29990661,9863
12499,80,30023465,4,0,246,0,66,60,61.5,2.598076,...,3,20,0.0,0.0,0,0,1.500112e+07,14991796.00,29992920,9328


In [72]:
# train_df = pd.read_csv('train.csv')
train_df = pd.read_csv('train2.csv')
train_df = train_df.drop(' Fwd Header Length.1',axis = 1)
train_df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,38308,1,1,6,6,6,6,6.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,389,479,11,5,172,326,79,0,15.636364,31.449238,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,88,1095,10,6,3150,3150,1575,0,315.000000,632.561635,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,389,15206,17,12,3452,6660,1313,0,203.058824,425.778474,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,88,1092,9,6,3150,3152,1575,0,350.000000,694.509719,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692698,53,32215,4,2,112,152,28,28,28.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
692699,53,324,2,2,84,362,42,42,42.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
692700,58030,82,2,1,31,6,31,0,15.500000,21.920310,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
692701,53,1048635,6,2,192,256,32,32,32.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [73]:
train_df.loc[(train_df[' Label'] != 'BENIGN'), ' Label'] = 'ATTACK'
print("Class distribution: {}".format(Counter(train_df[' Label']))) ## 1:BENIGN, 0: ATTACK

Class distribution: Counter({'BENIGN': 440031, 'ATTACK': 252672})


In [74]:
le=LabelEncoder()

In [75]:
train_df[' Label']=le.fit_transform(train_df[' Label'])
print("Class distribution: {}".format(Counter(train_df[' Label']))) ## 1:BENIGN, 0: ATTACK

Class distribution: Counter({1: 440031, 0: 252672})


In [76]:
train_df = train_df.dropna()

In [77]:
TAG_MIN = train_df[train_df.columns].min()
TAG_MAX = train_df[train_df.columns].max()

In [78]:
def normalize(df):
    ndf = df.copy()
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]:
            ndf[c] = df[c] - TAG_MIN[c]
        else:
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c])
    return ndf

In [79]:
train_df = normalize(train_df[train_df.columns])
train_df = train_df.dropna()
train_df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.001222,3.192417e-04,0.000000,0.000004,0.000005,9.569378e-09,0.000242,0.002906,0.001293,0.000000,...,0.344262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.005940,4.000000e-06,0.000049,0.000018,0.000141,5.199362e-07,0.003183,0.000000,0.003369,0.004892,...,0.540984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.001344,9.133333e-06,0.000044,0.000022,0.002573,5.023923e-06,0.063457,0.000000,0.067877,0.098389,...,0.540984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.005940,1.267250e-04,0.000078,0.000044,0.002820,1.062201e-05,0.052901,0.000000,0.043756,0.066226,...,0.540984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.001344,9.108333e-06,0.000039,0.000022,0.002573,5.027113e-06,0.063457,0.000000,0.075419,0.108024,...,0.540984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692698,0.000809,2.684667e-04,0.000015,0.000007,0.000091,2.424242e-07,0.001128,0.013559,0.006033,0.000000,...,0.344262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
692699,0.000809,2.708333e-06,0.000005,0.000007,0.000069,5.773525e-07,0.001692,0.020339,0.009050,0.000000,...,0.344262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
692700,0.886130,6.916667e-07,0.000005,0.000004,0.000025,9.569378e-09,0.001249,0.000000,0.003340,0.003409,...,0.540984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
692701,0.000809,8.738633e-03,0.000025,0.000007,0.000157,4.082935e-07,0.001289,0.015496,0.006895,0.000000,...,0.344262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [80]:
train_df_x=train_df.drop([' Label'],axis=1)
train_df_y=train_df[' Label']

Class distribution: Counter({1.0: 439683, 0.0: 251723})


In [81]:
train_df_x, valid_df_x, train_df_y, valid_df_y = train_test_split(train_df_x, train_df_y, 
                                                                  test_size = 0.3,
                                                                  shuffle = True,
                                                                 stratify = train_df_y,
                                                                 random_state = 30)

In [82]:
rf=RandomForestClassifier()
rf.fit(train_df_x,train_df_y)
rf_pred=rf.predict(train_df_x)

In [83]:
print(accuracy_score(train_df_y, rf_pred))

0.9997045356871301


In [84]:
confusion_matrix(train_df_y, rf_pred)

array([[176204,      2],
       [   141, 307637]], dtype=int64)

In [85]:
rf_pred_valid = rf.predict(valid_df_x)
print(accuracy_score(valid_df_y,rf_pred_valid))

0.999455216900811


In [86]:
confusion_matrix(valid_df_y, rf_pred_valid)

array([[ 75502,     15],
       [    98, 131807]], dtype=int64)

In [87]:
fi = []
for i in range(len(train_df.columns)-1):
#     print((list(train_df.columns)[i],list(rf.feature_importances_)[i]))
    fi.append((list(train_df.columns)[i],round(list(rf.feature_importances_)[i]*100,2)))
fi

[(' Destination Port', 5.11),
 (' Flow Duration', 0.48),
 (' Total Fwd Packets', 0.95),
 (' Total Backward Packets', 1.64),
 ('Total Length of Fwd Packets', 1.39),
 (' Total Length of Bwd Packets', 0.46),
 (' Fwd Packet Length Max', 2.07),
 (' Fwd Packet Length Min', 1.38),
 (' Fwd Packet Length Mean', 1.83),
 (' Fwd Packet Length Std', 0.31),
 ('Bwd Packet Length Max', 3.81),
 (' Bwd Packet Length Min', 0.74),
 (' Bwd Packet Length Mean', 7.65),
 (' Bwd Packet Length Std', 7.16),
 ('Flow Bytes/s', 0.0),
 (' Flow Packets/s', 0.0),
 (' Flow IAT Mean', 3.92),
 (' Flow IAT Std', 0.53),
 (' Flow IAT Max', 2.26),
 (' Flow IAT Min', 0.21),
 ('Fwd IAT Total', 0.27),
 (' Fwd IAT Mean', 0.79),
 (' Fwd IAT Std', 1.67),
 (' Fwd IAT Max', 0.85),
 (' Fwd IAT Min', 0.45),
 ('Bwd IAT Total', 0.07),
 (' Bwd IAT Mean', 0.14),
 (' Bwd IAT Std', 0.1),
 (' Bwd IAT Max', 0.19),
 (' Bwd IAT Min', 0.14),
 ('Fwd PSH Flags', 0.01),
 (' Bwd PSH Flags', 0.0),
 (' Fwd URG Flags', 0.0),
 (' Bwd URG Flags', 0.0),
 

In [88]:
fi = sorted(fi,key = lambda x : x[1], reverse = True)
fi

[(' Bwd Packet Length Mean', 7.65),
 (' Bwd Packet Length Std', 7.16),
 (' Avg Bwd Segment Size', 6.6),
 (' Packet Length Mean', 6.11),
 (' Destination Port', 5.11),
 (' Average Packet Size', 5.02),
 (' Max Packet Length', 4.62),
 (' Packet Length Std', 4.16),
 (' Flow IAT Mean', 3.92),
 ('Bwd Packet Length Max', 3.81),
 (' Packet Length Variance', 3.8),
 (' Init_Win_bytes_backward', 2.81),
 (' Avg Fwd Segment Size', 2.62),
 ('Init_Win_bytes_forward', 2.59),
 (' Flow IAT Max', 2.26),
 (' Fwd Packet Length Max', 2.07),
 (' Idle Max', 1.9),
 (' Fwd Packet Length Mean', 1.83),
 (' Fwd IAT Std', 1.67),
 (' Total Backward Packets', 1.64),
 (' Bwd Header Length', 1.43),
 (' Min Packet Length', 1.4),
 ('Total Length of Fwd Packets', 1.39),
 (' Fwd Packet Length Min', 1.38),
 (' Subflow Fwd Bytes', 1.35),
 (' Subflow Bwd Packets', 1.34),
 (' Fwd Header Length', 1.23),
 ('Idle Mean', 0.99),
 (' Total Fwd Packets', 0.95),
 (' Idle Min', 0.93),
 (' Fwd IAT Max', 0.85),
 (' Fwd IAT Mean', 0.79),
 

In [90]:
colums_choosed = []
for i in range(len(fi)):
    if fi[i][1] >= 1.0:
        colums_choosed.append(fi[i][0])
colums_choosed

[' Bwd Packet Length Mean',
 ' Bwd Packet Length Std',
 ' Avg Bwd Segment Size',
 ' Packet Length Mean',
 ' Destination Port',
 ' Average Packet Size',
 ' Max Packet Length',
 ' Packet Length Std',
 ' Flow IAT Mean',
 'Bwd Packet Length Max',
 ' Packet Length Variance',
 ' Init_Win_bytes_backward',
 ' Avg Fwd Segment Size',
 'Init_Win_bytes_forward',
 ' Flow IAT Max',
 ' Fwd Packet Length Max',
 ' Idle Max',
 ' Fwd Packet Length Mean',
 ' Fwd IAT Std',
 ' Total Backward Packets',
 ' Bwd Header Length',
 ' Min Packet Length',
 'Total Length of Fwd Packets',
 ' Fwd Packet Length Min',
 ' Subflow Fwd Bytes',
 ' Subflow Bwd Packets',
 ' Fwd Header Length']

In [91]:
train_df_x = train_df_x[colums_choosed]
valid_df_x = valid_df_x[colums_choosed]

In [92]:
rf=RandomForestClassifier()
rf.fit(train_df_x,train_df_y)
rf_pred=rf.predict(train_df_x)

In [93]:
print(accuracy_score(train_df_y, rf_pred))

0.9997313960792092


In [94]:
rf_pred_valid=rf.predict(valid_df_x)
print(accuracy_score(valid_df_y, rf_pred_valid))

0.9995034277945445


In [128]:
confusion_matrix(train_df_y, rf_pred)

array([[ 97684,      2],
       [    49, 127976]], dtype=int64)

In [95]:
test_df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,80,26870384,5,0,659,0,413,60,131.8,140.619202,...,4,20,0.0,0.0,0,0,7.291958e+06,10295440.45,21851909,9971
1,43908,372198,5,0,486,0,214,66,97.2,58.482134,...,1,20,0.0,0.0,0,0,1.575985e+05,158.50,157757,157440
2,80,27580343,5,0,827,0,581,60,165.4,207.812993,...,4,20,0.0,0.0,0,0,7.517380e+06,10594394.43,22500101,7682
3,80,27248690,5,0,627,0,381,60,125.4,127.821125,...,4,20,0.0,0.0,0,0,7.409746e+06,10452004.13,22191106,7534
4,80,28830565,5,0,791,0,545,60,158.2,193.413960,...,4,20,0.0,0.0,0,0,7.927495e+06,11165319.06,23717619,9795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12496,80,30027649,4,0,246,0,66,60,61.5,2.598076,...,3,20,0.0,0.0,0,0,1.500093e+07,14990575.50,29991510,10359
12497,80,2119685,4,0,246,0,66,60,61.5,2.598076,...,3,20,0.0,0.0,0,0,1.059838e+06,1047813.50,2107651,12024
12498,80,30021918,4,0,246,0,66,60,61.5,2.598076,...,3,20,0.0,0.0,0,0,1.500026e+07,14990399.00,29990661,9863
12499,80,30023465,4,0,246,0,66,60,61.5,2.598076,...,3,20,0.0,0.0,0,0,1.500112e+07,14991796.00,29992920,9328


In [98]:
TAG_MIN = train_df[train_df.columns].min()
TAG_MAX = train_df[train_df.columns].max()

In [96]:
TAG_MIN = test_df[test_df.columns].min()
TAG_MAX = test_df[test_df.columns].max()

In [97]:
test_df = normalize(test_df[test_df.columns])
test_df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.000000,0.424051,0.000836,0.0,0.000071,0.0,0.005411,0.000000,0.012815,0.011682,...,0.000836,0.0,0.0,0.0,0.0,0.0,0.423030,0.597677,0.634065,0.002007
1,0.739339,0.005874,0.000836,0.0,0.000050,0.0,0.002360,0.428571,0.006640,0.004859,...,0.000209,0.0,0.0,0.0,0.0,0.0,0.009143,0.000009,0.004578,0.031693
2,0.000000,0.435255,0.000836,0.0,0.000091,0.0,0.007986,0.000000,0.018813,0.017265,...,0.000836,0.0,0.0,0.0,0.0,0.0,0.436108,0.615032,0.652874,0.001546
3,0.000000,0.430022,0.000836,0.0,0.000067,0.0,0.004920,0.000000,0.011673,0.010619,...,0.000836,0.0,0.0,0.0,0.0,0.0,0.429864,0.606766,0.643908,0.001517
4,0.000000,0.454986,0.000836,0.0,0.000087,0.0,0.007434,0.000000,0.017527,0.016069,...,0.000836,0.0,0.0,0.0,0.0,0.0,0.459900,0.648176,0.688202,0.001972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12496,0.000000,0.473877,0.000627,0.0,0.000022,0.0,0.000092,0.000000,0.000268,0.000216,...,0.000627,0.0,0.0,0.0,0.0,0.0,0.870254,0.870242,0.870248,0.002085
12497,0.000000,0.033452,0.000627,0.0,0.000022,0.0,0.000092,0.000000,0.000268,0.000216,...,0.000627,0.0,0.0,0.0,0.0,0.0,0.061485,0.060828,0.061157,0.002420
12498,0.000000,0.473787,0.000627,0.0,0.000022,0.0,0.000092,0.000000,0.000268,0.000216,...,0.000627,0.0,0.0,0.0,0.0,0.0,0.870215,0.870232,0.870223,0.001985
12499,0.000000,0.473811,0.000627,0.0,0.000022,0.0,0.000092,0.000000,0.000268,0.000216,...,0.000627,0.0,0.0,0.0,0.0,0.0,0.870265,0.870313,0.870289,0.001878


In [98]:
test_df=test_df[colums_choosed]
test_df

,Bwd Packet Length Mean,Bwd Packet Length Std,Avg Bwd Segment Size,Packet Length Mean,Destination Port,Average Packet Size,Max Packet Length,Packet Length Std,Flow IAT Mean,Bwd Packet Length Max,...,Fwd Packet Length Mean,Fwd IAT Std,Total Backward Packets,Bwd Header Length,Min Packet Length,Total Length of Fwd Packets,Fwd Packet Length Min,Subflow Fwd Bytes,Subflow Bwd Packets,Fwd Header Length
0,0.0,0.0,0.0,0.012815,0.000000,0.012815,0.005411,0.011682,0.381578,0.0,...,0.012815,0.598685,0.0,0.0,0.000000,0.000071,0.000000,0.000071,0.0,0.000836
1,0.0,0.0,0.0,0.006640,0.739339,0.006640,0.002360,0.004859,0.005285,0.0,...,0.006640,0.004509,0.0,0.0,0.428571,0.000050,0.428571,0.000050,0.0,0.000836
2,0.0,0.0,0.0,0.018813,0.000000,0.018813,0.007986,0.017265,0.391660,0.0,...,0.018813,0.617376,0.0,0.0,0.000000,0.000091,0.000000,0.000091,0.0,0.000836
3,0.0,0.0,0.0,0.011673,0.000000,0.011673,0.004920,0.010619,0.386950,0.0,...,0.011673,0.608439,0.0,0.0,0.000000,0.000067,0.000000,0.000067,0.0,0.000836
4,0.0,0.0,0.0,0.017527,0.000000,0.017527,0.007434,0.016069,0.409414,0.0,...,0.017527,0.651883,0.0,0.0,0.000000,0.000087,0.000000,0.000087,0.0,0.000836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12496,0.0,0.0,0.0,0.000268,0.000000,0.000268,0.000092,0.000216,0.568552,0.0,...,0.000268,0.942352,0.0,0.0,0.000000,0.000022,0.000000,0.000022,0.0,0.000627
12497,0.0,0.0,0.0,0.000268,0.000000,0.000268,0.000092,0.000216,0.040135,0.0,...,0.000268,0.066041,0.0,0.0,0.000000,0.000022,0.000000,0.000022,0.0,0.000627
12498,0.0,0.0,0.0,0.000268,0.000000,0.000268,0.000092,0.000216,0.568443,0.0,...,0.000268,0.942379,0.0,0.0,0.000000,0.000022,0.000000,0.000022,0.0,0.000627
12499,0.0,0.0,0.0,0.000268,0.000000,0.000268,0.000092,0.000216,0.568472,0.0,...,0.000268,0.942436,0.0,0.0,0.000000,0.000022,0.000000,0.000022,0.0,0.000627


In [99]:
test_pred = rf.predict(test_df)
print("Class distribution: {}".format(Counter(test_pred))) ## 1:BENIGN, 0: ATTACK

Class distribution: Counter({1.0: 12166, 0.0: 335})


In [103]:
test_pred = list(test_pred)
for i in range(len(test_pred)):
    if test_pred[i] == 1.0:
        test_pred[i] = 'BENIGN'
    else:
        test_pred[i] = 'ATTACK'
Counter(test_pred)

Counter({'BENIGN': 12166, 'ATTACK': 335})

In [114]:
answer = pd.DataFrame({'index' : np.arange(len(test_pred)),
                       'answer' : test_pred})
answer.to_csv('RF_answer.csv', 
              index = False)
answer

,index,answer
0,0,BENIGN
1,1,BENIGN
2,2,BENIGN
3,3,BENIGN
4,4,BENIGN
...,...,...
12496,12496,BENIGN
12497,12497,BENIGN
12498,12498,BENIGN
12499,12499,BENIGN


In [116]:
df = pd.read_csv('RF_answer.csv', index = False)
df

TypeError: read_csv() got an unexpected keyword argument 'index'